In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# importing the module 
import json 
import pandas as pd
  
# Opening JSON file 
with open('cell.json') as json_file: 
    data = json.load(json_file) 
  
    # Print the type of data variable 
    print("Type:", type(data)) 
  
    df=pd.DataFrame(data) # creating a dataframe

Type: <class 'dict'>


In [4]:
df.head()

Rating  Rev_verify  Review_Date          IC             Prod_meta  \
0       5        True   09 1, 2015  B009XD5TPQ                  None   
1       5        True   01 9, 2016  B016MF3P3K                  None   
2       5        True  04 21, 2013  B008DC8N5G                  None   
3       3        True  02 27, 2013  B0089CH3TM  {'Color:': ' Green'}   
4       4        True  12 19, 2013  B00AKZWGAC                  None   

    Reviewer_Name                                             Review  \
0  Sunny  Zoeller  Bought it for my husband. He's very happy with it   
1   Denise Lesley  Great screen protector.  Doesn't even seem as ...   
2            Emir  Saved me lots of money! it's not gorilla glass...   
3           Alyse  The material and fit is very nice, but the col...   
4         TechGuy  This last me about 3 days till i have to charg...   

                                Rev_summ  Review_timestamp Useful Prod_img  
0                   He's very happy with        1441065600   None     None  
1                             Five Stars        1452297600   None     None  
2  As long as you know how to put it on!        1366502400   None     None  
3                      Good case overall        1361923200      3     None  
4                        Awesome Battery        1387411200   None     None

In [ ]:
# Inspecting into IC column
df['IC'].value_counts()

B00VH88CJ0    1127
B005NF5NTK    1106
B019PV2I3G    1033
B00X5RV14Y    1018
B00P7N0320    1011
              ... 
B00BY5GVPI       1
B009525YUK       1
B01CVOV3T4       1
B0010S8S3C       1
B0177V64O0       1
Name: IC, Length: 48134, dtype: int64

In [ ]:
pd.set_option('display.max_columns',100)

In [ ]:
# checking how many unique products we have
df['IC'].nunique()

48134

In [ ]:
df.shape

(760450, 11)

In [ ]:
# There are 48k+ products listed here 

In [ ]:
# Initially, we want to use only 2 cols, i.e. IC and Review and we want to group the sentences of reviews by ICs

In [5]:
unique_ic=df['IC'].unique()

In [ ]:
len(unique_ic)

48134

# BASIC PLANNING

In [ ]:
df[df['IC']==unique_ic[0]]['Review']

0         Bought it for my husband. He's very happy with it
208513    Good product.  Does exactly what the descripti...
372808    This is a very nice case for medium to larger ...
380807    Appears to be excellent product. Fits the phon...
395309    Great fit for iPhone 5 with protective case on...
419184    The quality of this case feels nice and solid....
535227    Good phone "holster" with belt loops and clip....
539885    [July 9, 2013] After I picked up " Body Glove ...
552097    Better than I would expect, been using it dail...
557748    The case is as it was explained.  However, it ...
673999    This case is quite durable. Excellent price. A...
752862                                        Happy husband
Name: Review, dtype: object

In [ ]:
df[df['IC']==unique_ic[0]]['Review'][0]

"Bought it for my husband. He's very happy with it"

In [ ]:
# Now, we'll use for loops to combine all the reviews corresponding to same ICs

In [ ]:
# But first, lets take all the text in one of the ICs and combine it and make a model on it

In [ ]:
df[df['IC']=='B00VH88CJ0']['Review']

334                                        good car charger
1175      I love this charger. Like everything I have pu...
2897      Anker is setting a new standard for usb device...
3814      Like other Anker products, a dependable and ef...
3878      This charger is working great in my wife's car...
                                ...                        
758405    Works great in my car. Feels lighter than expe...
758578    seller (or Anker) accidentally sent me one of ...
758821                                                Good!
759382                                 Works as advertised.
759669                                             GREAT!!!
Name: Review, Length: 1127, dtype: object

In [ ]:
len(df[df['IC']=='B00VH88CJ0'])
df[df['IC']=='B00VH88CJ0']['Review'][2]

'Anker is setting a new standard for usb devices. Completely satisfied with this product.'

In [ ]:
text=''
for i in range(len(df[df['IC']=='B00VH88CJ0'])):
  if df[df['IC']=='B00VH88CJ0']['Review'][i] is None:
    continue
  else:
    text=text+ df[df['IC']=='B00VH88CJ0']['Review'][i] + '\n'

In [ ]:
# Now, let's build a simple topic model on this piece of text

In [6]:
import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy
import pandas as pd

from gensim.models import LdaModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import re

import warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now
%matplotlib inline

In [7]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [8]:
def clean(sent):
    sent=decontracted(sent)
    sent=re.sub(r'https?://\S+', '', sent)
    sent=re.sub(r'[\w\.-]+@[\w\.-]+', '', sent)
    sent = re.sub("\S*\d\S*", "", sent)
    sent=re.sub(r'[^A-Za-z0-9]+', ' ', sent)
    sent=sent.lower()
    sent = ' '.join(e for e in sent.split() if len(e)>2 )
    sent=sent.translate(str.maketrans('', '', '"#%&\'()*+,-./:;<=>@[\\]^_`{|}~'))

    return sent

In [9]:
nlp = spacy.load("en")

In [ ]:
cleaned=[]
for i in text.split('\n'):
    sent=nlp(clean(i))
    cleaned.append(sent)
print(len(cleaned))

1573


In [ ]:
article, texts = [], []
for doc in cleaned:
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article
        if not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)
    texts.append(article)
    article=[]

In [ ]:
len(texts)

1573

In [ ]:
bigram = gensim.models.Phrases(texts)
texts = [bigram[line] for line in texts]
texts[4][:20]

['anker',
 'set',
 'new',
 'standard',
 'usb',
 'device',
 'completely',
 'satisfied',
 'product']

In [ ]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=30, random_state=0)
topics=ldamodel.print_topics(num_words=15)
topics

[(0,
  '0.044*"charge" + 0.038*"charger" + 0.025*"anker" + 0.025*"work" + 0.023*"car" + 0.021*"great" + 0.019*"product" + 0.016*"phone" + 0.014*"good" + 0.013*"fast" + 0.011*"device" + 0.009*"buy" + 0.007*"power" + 0.007*"use" + 0.007*"like"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=50, random_state=0)
topics=ldamodel.print_topics(num_words=15)
topics

[(0,
  '0.044*"charge" + 0.038*"charger" + 0.025*"anker" + 0.025*"work" + 0.023*"car" + 0.021*"great" + 0.019*"product" + 0.016*"phone" + 0.014*"good" + 0.013*"fast" + 0.011*"device" + 0.009*"buy" + 0.007*"use" + 0.007*"power" + 0.007*"like"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=40, random_state=0)
topics=ldamodel.print_topics(num_words=20)
topics
# the words here are a good representative of all the reviews in the text
# we'll use 20 words to represent the summary of every set of reviews

[(0,
  '0.044*"charge" + 0.038*"charger" + 0.025*"anker" + 0.025*"work" + 0.023*"car" + 0.021*"great" + 0.019*"product" + 0.016*"phone" + 0.014*"good" + 0.013*"fast" + 0.011*"device" + 0.009*"buy" + 0.007*"use" + 0.007*"power" + 0.007*"like" + 0.007*"time" + 0.007*"port" + 0.007*"quality" + 0.006*"cable" + 0.006*"usb"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=50, random_state=0)
topics=ldamodel.print_topics(num_words=25)
topics

[(0,
  '0.044*"charge" + 0.038*"charger" + 0.025*"anker" + 0.025*"work" + 0.023*"car" + 0.021*"great" + 0.019*"product" + 0.016*"phone" + 0.014*"good" + 0.013*"fast" + 0.011*"device" + 0.009*"buy" + 0.007*"power" + 0.007*"use" + 0.007*"like" + 0.007*"time" + 0.007*"port" + 0.007*"quality" + 0.006*"cable" + 0.006*"usb" + 0.006*"love" + 0.006*"iphone" + 0.006*"plug" + 0.005*"need" + 0.004*"get"')]

In [ ]:
# Now, let's try the approach above on some of the ICs having less amount of data to see how it performs

In [ ]:
len(df[df['IC']==unique_ic[50]])

55

In [ ]:
df[df['IC']==unique_ic[50]]['Review']

50                                                    great
2007                All worked as described and as expected
9772         Works fine and has withstood the test of time.
65022                                            five stars
73407     don't like it much. as soon as I got it, I thr...
93586     Bought this for a friend and She loves the Case..
98850                         Great low profile rugged case
101522    Rugged case adn the built in stand is nice to ...
126165                              Nice, use it every day.
132257       Very rugged. Works great.  I like this product
136801    its the best cover for a samsung s3 mini cell ...
150396               Excellent condition fits true to size.
153244                                              perfect
181365                                  Prefect and on time
188056    this one is a little too bulky.  it stuck to t...
215788                            Heat product just as said
228029                    Fits phone per

In [ ]:
# there are only 55 reviews corresponding to this particular IC. Let's use this

In [ ]:
text=''
for i in range(len(df[df['IC']==unique_ic[50]])):
  if df[df['IC']==unique_ic[50]]['Review'][i] is None:
    continue
  else:
    text=text+ df[df['IC']==unique_ic[50]]['Review'][i] + '\n'

In [ ]:
# write this text to a txt file
f = open("demo_text.txt", "w")
f.write(text)
f.close()

In [ ]:
cleaned=[]
for i in text.split('\n'):
    sent=nlp(clean(i))
    cleaned.append(sent)
print(len(cleaned))

article, texts = [], []
for doc in cleaned:
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article
        if not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)
    texts.append(article)
    article=[]

print(len(texts))

bigram = gensim.models.Phrases(texts)
texts = [bigram[line] for line in texts]

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=40, random_state=0)
topics=ldamodel.print_topics(num_words=20)
topics

67
67


[(0,
  '0.042*"case" + 0.039*"phone" + 0.021*"great" + 0.016*"button" + 0.013*"love" + 0.012*"protect" + 0.011*"hard" + 0.011*"like" + 0.010*"get" + 0.010*"nice" + 0.009*"cover" + 0.009*"easy" + 0.009*"good" + 0.009*"buy" + 0.009*"stand" + 0.009*"fit" + 0.009*"work" + 0.009*"press" + 0.008*"time" + 0.008*"product"')]

In [ ]:
# The words are a good representation of the entire corpus of reviews corresponding to this IC

In [ ]:
# trying to extract just words
for i in topics:
  string=i[1]
  print(type(string))
  print(string)
  string=string.replace(' ','').replace('*','').replace('+','').replace('"','')
  print(string)
  string = re.sub("\d+", "", string)
  string=string.replace('.',',')
  string=string[1:]
  print(string)

<class 'str'>
0.042*"case" + 0.039*"phone" + 0.021*"great" + 0.016*"button" + 0.013*"love" + 0.012*"protect" + 0.011*"hard" + 0.011*"like" + 0.010*"get" + 0.010*"nice" + 0.009*"cover" + 0.009*"easy" + 0.009*"good" + 0.009*"buy" + 0.009*"stand" + 0.009*"fit" + 0.009*"work" + 0.009*"press" + 0.008*"time" + 0.008*"product"
0.042case0.039phone0.021great0.016button0.013love0.012protect0.011hard0.011like0.010get0.010nice0.009cover0.009easy0.009good0.009buy0.009stand0.009fit0.009work0.009press0.008time0.008product
case,phone,great,button,love,protect,hard,like,get,nice,cover,easy,good,buy,stand,fit,work,press,time,product


In [ ]:
string

'case,phone,great,button,love,protect,hard,like,get,nice,cover,easy,good,buy,stand,fit,work,press,time,product'

In [ ]:
# There could be two more approaches to this. 
# 1. Let's combine all the reviews with all the review summaries product wise and then use LDA on that data.
# 2. Let's build a seperate model of LDA on Reviews to predict 20 words and a seperate model on Review summaries to predict 3 to 5 words.

In [ ]:
# First, lets try approach second
# As we have already built a model on Review data seperately, here we'll only use the review_summary data

In [ ]:
text=''
for i in range(len(df[df['IC']==unique_ic[50]])):
  if df[df['IC']==unique_ic[50]]['Rev_summ'][i] is None:
    continue
  else:
    text=text+ df[df['IC']==unique_ic[50]]['Rev_summ'][i] + '\n'

cleaned=[]
for i in text.split('\n'):
    sent=nlp(clean(i))
    cleaned.append(sent)
print(len(cleaned))

article, texts = [], []
for doc in cleaned:
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article
        if not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)
    texts.append(article)
    article=[]

print(len(texts))

bigram = gensim.models.Phrases(texts)
texts = [bigram[line] for line in texts]

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=40, random_state=0)
topics=ldamodel.print_topics(num_words=20)
topics

56
56


[(0,
  '0.127*"star" + 0.061*"case" + 0.055*"great" + 0.042*"good" + 0.024*"like" + 0.024*"nice" + 0.024*"price" + 0.024*"love" + 0.018*"protection" + 0.018*"durable" + 0.018*"awesome" + 0.018*"cover" + 0.018*"quality" + 0.018*"value" + 0.018*"product" + 0.018*"work" + 0.018*"money" + 0.018*"phone" + 0.012*"yes" + 0.012*"hard"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=30, random_state=0)
topics=ldamodel.print_topics(num_words=5)
topics

[(0,
  '0.127*"star" + 0.061*"case" + 0.055*"great" + 0.042*"good" + 0.024*"nice"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=30, random_state=0)
topics=ldamodel.print_topics(num_words=10)
topics

[(0,
  '0.127*"star" + 0.061*"case" + 0.055*"great" + 0.042*"good" + 0.024*"nice" + 0.024*"price" + 0.024*"love" + 0.024*"like" + 0.018*"phone" + 0.018*"durable"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=20, random_state=0)
topics=ldamodel.print_topics(num_words=3)
topics

[(0, '0.127*"star" + 0.061*"case" + 0.055*"great"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=2, id2word=dictionary, passes=20, random_state=0)
topics=ldamodel.print_topics(num_words=5)
topics

[(0,
  '0.237*"star" + 0.074*"good" + 0.029*"value" + 0.029*"awesome" + 0.028*"cover"'),
 (1,
  '0.119*"case" + 0.108*"great" + 0.044*"price" + 0.044*"like" + 0.033*"love"')]

In [ ]:
# Till now, we were using single topic with multiple words (num_topics=1 and num_words=10 or so)
# Rather than this, let's try num_topics=10 and num_words=1

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary, passes=50, random_state=0)
topics=ldamodel.print_topics(num_words=1)
topics

[(0, '0.602*"star"'),
 (1, '0.169*"like"'),
 (2, '0.303*"great"'),
 (3, '0.172*"pleased"'),
 (4, '0.131*"bulky"'),
 (5, '0.250*"good"'),
 (6, '0.121*"value"'),
 (7, '0.098*"case"'),
 (8, '0.211*"case"'),
 (9, '0.106*"phone"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=4, id2word=dictionary, passes=30, random_state=0)
topics=ldamodel.print_topics(num_words=2)
topics
# This one is a good representative of all the review summaries

[(0, '0.409*"star" + 0.126*"good"'),
 (1, '0.172*"case" + 0.064*"great"'),
 (2, '0.126*"great" + 0.056*"price"'),
 (3, '0.105*"awesome" + 0.058*"sturdy"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary, passes=30, random_state=0)
topics=ldamodel.print_topics(num_words=2)
topics

[(0, '0.369*"star" + 0.077*"good"'),
 (1, '0.206*"case" + 0.067*"great"'),
 (2, '0.180*"great" + 0.065*"price"'),
 (3, '0.076*"work" + 0.076*"fine"'),
 (4, '0.085*"case" + 0.047*"love"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=4, id2word=dictionary, passes=30, random_state=0)
topics=ldamodel.print_topics(num_words=3)
topics

[(0, '0.409*"star" + 0.126*"good" + 0.045*"value"'),
 (1, '0.172*"case" + 0.064*"great" + 0.060*"like"'),
 (2, '0.126*"great" + 0.056*"price" + 0.056*"product"'),
 (3, '0.105*"awesome" + 0.058*"sturdy" + 0.058*"functional"')]

In [ ]:
# Now, Lets apply the first approach. Lets combine data of Review and Review_summary and then apply LDA on that.

In [ ]:
text=''
for i in range(len(df[df['IC']==unique_ic[50]])):
  if (df[df['IC']==unique_ic[50]]['Rev_summ'][i] is None) or (df[df['IC']==unique_ic[50]]['Review'][i] is None):
    continue
  else:
    text=text+ df[df['IC']==unique_ic[50]]['Rev_summ'][i] + " " + df[df['IC']==unique_ic[50]]['Review'][i] + '\n'

# print(text)

cleaned=[]
for i in text.split('\n'):
    sent=nlp(clean(i))
    cleaned.append(sent)
print(len(cleaned))

article, texts = [], []
for doc in cleaned:
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article
        if not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)
    texts.append(article)
    article=[]

print(len(texts))

bigram = gensim.models.Phrases(texts)
texts = [bigram[line] for line in texts]

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary, passes=60, random_state=0)
topics=ldamodel.print_topics(num_words=2)
topics

67
67


[(0, '0.095*"case" + 0.095*"great"'),
 (1, '0.078*"like" + 0.021*"get"'),
 (2, '0.059*"good" + 0.021*"thank"'),
 (3, '0.065*"case" + 0.030*"good"'),
 (4, '0.136*"star" + 0.057*"work"'),
 (5, '0.061*"case" + 0.061*"phone"'),
 (6, '0.060*"phone" + 0.040*"case"'),
 (7, '0.034*"price" + 0.034*"great"'),
 (8, '0.061*"phone" + 0.031*"price"'),
 (9, '0.048*"star" + 0.039*"great"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary, passes=60, random_state=0)
topics=ldamodel.print_topics(num_words=1)
topics

[(0, '0.064*"case"'),
 (1, '0.034*"phone"'),
 (2, '0.030*"functional"'),
 (3, '0.051*"case"'),
 (4, '0.050*"star"')]

In [ ]:
# Mixing both the data might not be a good idea, becuase results are same as we got on rev_summ data alone
# Maybe we should try to build a single topic on combination of data like we did earlier on Review data

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=60, random_state=0)
topics=ldamodel.print_topics(num_words=15)
topics

[(0,
  '0.045*"case" + 0.036*"phone" + 0.026*"great" + 0.024*"star" + 0.015*"love" + 0.014*"good" + 0.014*"button" + 0.014*"like" + 0.012*"protect" + 0.012*"nice" + 0.011*"hard" + 0.010*"price" + 0.010*"cover" + 0.010*"work" + 0.009*"buy"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=80, random_state=0)
topics=ldamodel.print_topics(num_words=22)
topics

[(0,
  '0.045*"case" + 0.036*"phone" + 0.026*"great" + 0.024*"star" + 0.015*"love" + 0.014*"good" + 0.014*"button" + 0.014*"like" + 0.012*"protect" + 0.012*"nice" + 0.011*"hard" + 0.010*"price" + 0.010*"cover" + 0.010*"work" + 0.009*"buy" + 0.009*"product" + 0.009*"get" + 0.008*"stand" + 0.008*"easy" + 0.008*"fit" + 0.008*"press" + 0.008*"time"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=2, id2word=dictionary, passes=80, random_state=0)
topics=ldamodel.print_topics(num_words=10)
topics

[(0,
  '0.059*"case" + 0.040*"great" + 0.039*"phone" + 0.038*"star" + 0.025*"love" + 0.016*"price" + 0.016*"protect" + 0.014*"product" + 0.014*"work" + 0.014*"nice"'),
 (1,
  '0.032*"phone" + 0.031*"button" + 0.027*"case" + 0.018*"hard" + 0.018*"like" + 0.017*"good" + 0.013*"press" + 0.012*"buy" + 0.011*"get" + 0.010*"place"')]

In [ ]:
# In above cell, we noticed that maybe num_topics=1 makes more sense than num_topics=10 or so

In [ ]:
# Lets try with Review data alone and build multiple topics 

In [ ]:
text=''
for i in range(len(df[df['IC']==unique_ic[50]])):
  if df[df['IC']==unique_ic[50]]['Review'][i] is None:
    continue
  else:
    text=text+ df[df['IC']==unique_ic[50]]['Review'][i] + '\n'

cleaned=[]
for i in text.split('\n'):
    sent=nlp(clean(i))
    cleaned.append(sent)
print(len(cleaned))

article, texts = [], []
for doc in cleaned:
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article
        if not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)
    texts.append(article)
    article=[]

print(len(texts))

bigram = gensim.models.Phrases(texts)
texts = [bigram[line] for line in texts]

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary, passes=40, random_state=0)
topics=ldamodel.print_topics(num_words=1)
topics

67
67


[(0, '0.081*"great"'),
 (1, '0.063*"case"'),
 (2, '0.055*"button"'),
 (3, '0.062*"love"'),
 (4, '0.069*"case"'),
 (5, '0.049*"pocket"'),
 (6, '0.055*"phone"'),
 (7, '0.067*"case"'),
 (8, '0.029*"like"'),
 (9, '0.063*"phone"')]

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=4, id2word=dictionary, passes=80, random_state=0)
topics=ldamodel.print_topics(num_words=3)
topics

[(0, '0.054*"case" + 0.041*"phone" + 0.032*"great"'),
 (1, '0.045*"case" + 0.034*"phone" + 0.015*"thank"'),
 (2, '0.045*"button" + 0.024*"case" + 0.023*"like"'),
 (3, '0.053*"phone" + 0.041*"case" + 0.035*"love"')]

# CONCLUSION

In [ ]:
# In all the iterations conducted above, we noticed that-
# Building two separate models on Review_summary and Review and then combining the tokens acheived from both doesn't make much sense.
# Instead, combine the data and make a single LDA model

# Secondly, using num_topics more than 2 results in reptition of lot of keywords. Therefore we'll use num_topics= 1 or 2  

# DATA VISUALISATION

In [ ]:
df.head()

Rating  Rev_verify  Review_Date          IC             Prod_meta  \
0       5        True   09 1, 2015  B009XD5TPQ                  None   
1       5        True   01 9, 2016  B016MF3P3K                  None   
2       5        True  04 21, 2013  B008DC8N5G                  None   
3       3        True  02 27, 2013  B0089CH3TM  {'Color:': ' Green'}   
4       4        True  12 19, 2013  B00AKZWGAC                  None   

    Reviewer_Name                                             Review  \
0  Sunny  Zoeller  Bought it for my husband. He's very happy with it   
1   Denise Lesley  Great screen protector.  Doesn't even seem as ...   
2            Emir  Saved me lots of money! it's not gorilla glass...   
3           Alyse  The material and fit is very nice, but the col...   
4         TechGuy  This last me about 3 days till i have to charg...   

                                Rev_summ  Review_timestamp Useful Prod_img  
0                   He's very happy with        1441065600   None     None  
1                             Five Stars        1452297600   None     None  
2  As long as you know how to put it on!        1366502400   None     None  
3                      Good case overall        1361923200      3     None  
4                        Awesome Battery        1387411200   None     None

In [ ]:
df.shape

(760450, 11)

In [ ]:
df.columns

Index(['Rating', 'Rev_verify', 'Review_Date', 'IC', 'Prod_meta',
       'Reviewer_Name', 'Review', 'Rev_summ', 'Review_timestamp', 'Useful',
       'Prod_img'],
      dtype='object')

In [ ]:
df.describe()

Rating  Review_timestamp
count  760450.000000      7.604500e+05
mean        4.221357      1.440417e+09
std         1.232186      4.563251e+07
min         1.000000      1.035331e+09
25%         4.000000      1.416355e+09
50%         5.000000      1.444349e+09
75%         5.000000      1.470442e+09
max         5.000000      1.538438e+09

In [ ]:
# Unique products
df['IC'].nunique()

48134

In [ ]:
df['IC'].value_counts()[[0]]

B00VH88CJ0    1127
Name: IC, dtype: int64

In [ ]:
# Product having product code='B00VH88CJ0' have the maximum number of reviews available

In [ ]:
df['IC'].value_counts().iloc[-5:]

B003YTR4AI    1
B005MN7C92    1
B0016JB9OK    1
B008GXPI8W    1
B00P4US7QK    1
Name: IC, dtype: int64

In [ ]:
# These are products which have least number of reviews available

In [ ]:
# how many such products are available in our data, for which we have only one review
(df['IC'].value_counts()==1).sum()

427

In [10]:
# dataframe telling us about the minimum rating product wise
df_min_rating=df.groupby(by='IC',)['Rating'].min()
df_min_rating.head()

IC
7508492919    1
7532385086    1
7887421268    1
8199900164    2
8288853439    1
Name: Rating, dtype: int64

In [11]:
# dataframe telling us about the maximum rating product wise
df_max_rating=df.groupby(by='IC')['Rating'].max()
df_max_rating.head()

IC
7508492919    5
7532385086    5
7887421268    5
8199900164    5
8288853439    5
Name: Rating, dtype: int64

In [12]:
# dataframe telling us about the average rating product wise
df_avg_rating=df.groupby(by='IC')['Rating'].mean()
df_avg_rating.head()

IC
7508492919    4.300000
7532385086    3.285714
7887421268    3.538462
8199900164    3.750000
8288853439    3.818182
Name: Rating, dtype: float64

In [13]:
df.isnull().sum()

Rating                   0
Rev_verify               0
Review_Date              0
IC                       0
Prod_meta           352624
Reviewer_Name           91
Review                 530
Rev_summ               355
Review_timestamp         0
Useful              698250
Prod_img            742256
dtype: int64

In [ ]:
# More than half of the items don't have Prod_meta, so drop the columns
# Remove all the rows which don't have Review or Review summary
# Almost 90% of the values in Useful and Prod_img are missing, therefore drop those columns

In [14]:
df=df.drop(['Prod_meta', 'Useful', 'Prod_img'], axis=1)
df.head()

Rating  Rev_verify  Review_Date          IC   Reviewer_Name  \
0       5        True   09 1, 2015  B009XD5TPQ  Sunny  Zoeller   
1       5        True   01 9, 2016  B016MF3P3K   Denise Lesley   
2       5        True  04 21, 2013  B008DC8N5G            Emir   
3       3        True  02 27, 2013  B0089CH3TM           Alyse   
4       4        True  12 19, 2013  B00AKZWGAC         TechGuy   

                                              Review  \
0  Bought it for my husband. He's very happy with it   
1  Great screen protector.  Doesn't even seem as ...   
2  Saved me lots of money! it's not gorilla glass...   
3  The material and fit is very nice, but the col...   
4  This last me about 3 days till i have to charg...   

                                Rev_summ  Review_timestamp  
0                   He's very happy with        1441065600  
1                             Five Stars        1452297600  
2  As long as you know how to put it on!        1366502400  
3                      Good case overall        1361923200  
4                        Awesome Battery        1387411200

In [15]:
# Review_timestamp and Review date and Name don't have any role for our given task
# Lets drop them as well
df=df.drop(['Reviewer_Name', 'Review_Date', 'Review_timestamp'], axis=1)
df.head()

Rating  Rev_verify          IC  \
0       5        True  B009XD5TPQ   
1       5        True  B016MF3P3K   
2       5        True  B008DC8N5G   
3       3        True  B0089CH3TM   
4       4        True  B00AKZWGAC   

                                              Review  \
0  Bought it for my husband. He's very happy with it   
1  Great screen protector.  Doesn't even seem as ...   
2  Saved me lots of money! it's not gorilla glass...   
3  The material and fit is very nice, but the col...   
4  This last me about 3 days till i have to charg...   

                                Rev_summ  
0                   He's very happy with  
1                             Five Stars  
2  As long as you know how to put it on!  
3                      Good case overall  
4                        Awesome Battery

In [16]:
# Lets drop all the rows which have Nan in either Review or Review_summary
df.isnull().sum()

Rating          0
Rev_verify      0
IC              0
Review        530
Rev_summ      355
dtype: int64

In [17]:
df.dropna(0, inplace=True)

In [18]:
df.shape

(759602, 5)

In [ ]:
# Lets check the number of unique products once again
df['IC'].nunique()

48133

In [19]:
# Redefining the variable
unique_ic=df['IC'].unique()

In [ ]:
# The number of unique products decreased by one

# LDA MODEL

In [ ]:
text=''
for i in range(len(df[df['IC']==unique_ic[50]])):
  if (df[df['IC']==unique_ic[50]]['Rev_summ'][i] is None) or (df[df['IC']==unique_ic[50]]['Review'][i] is None):
    continue
  else:
    text=text+ df[df['IC']==unique_ic[50]]['Rev_summ'][i] + " " + df[df['IC']==unique_ic[50]]['Review'][i] + '\n'

# print(text)

cleaned=[]
for i in text.split('\n'):
    sent=nlp(clean(i))
    cleaned.append(sent)
print(len(cleaned))

article, texts = [], []
for doc in cleaned:
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article
        if not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)
    texts.append(article)
    article=[]

print(len(texts))

bigram = gensim.models.Phrases(texts)
texts = [bigram[line] for line in texts]

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=50, random_state=0)
topics=ldamodel.print_topics(num_words=20)
topics

67
67


[(0,
  '0.045*"case" + 0.036*"phone" + 0.026*"great" + 0.024*"star" + 0.015*"love" + 0.014*"good" + 0.014*"button" + 0.014*"like" + 0.012*"protect" + 0.012*"nice" + 0.011*"hard" + 0.010*"price" + 0.010*"cover" + 0.010*"work" + 0.009*"buy" + 0.009*"product" + 0.009*"get" + 0.008*"press" + 0.008*"fit" + 0.008*"stand"')]

In [ ]:
# The model seems sensible
# Now, we'll use the same model on each group of reviews (combined product-wise)

In [ ]:
# We'll create a new output data frame with columns-
# # IC (unique code for every product)
# Max_Rating
# Avg_Rating
# Min_Rating
# Keywords (String type)

In [20]:
data={'IC':[' ' for i in range(len(unique_ic))], 'Max_rating':[0 for i in range(len(unique_ic))], 'Min_rating': [0 for i in range(len(unique_ic))], 
      'Avg_rating': [0 for i in range(len(unique_ic))], 'Keywords': [' ' for i in range(len(unique_ic))]}

In [21]:
output=pd.DataFrame(data, columns=data.keys())

In [22]:
output.head()

IC  Max_rating  Min_rating  Avg_rating Keywords
0              0           0           0         
1              0           0           0         
2              0           0           0         
3              0           0           0         
4              0           0           0

In [ ]:
output.shape

(48133, 5)

In [ ]:
# We have created an outer structure of output dataframe

In [23]:
# # Now, let's edit the values of columns of Ratings
# First we'll generate a sample dataset with 200 samples
from tqdm import tqdm
for j in range(len(unique_ic[:200])):
  output['IC'].iloc[j]=unique_ic[j]
  output['Max_rating'].iloc[j]=df[df['IC']==unique_ic[j]]['Rating'].max()
  output['Min_rating'].iloc[j]=df[df['IC']==unique_ic[j]]['Rating'].min()
  output['Avg_rating'].iloc[j]=df[df['IC']==unique_ic[j]]['Rating'].mean()

In [24]:
# for j in tqdm(range(len(unique_ic))):
#   # output['IC'].iloc[j]=unique_ic[j]
#   output['Max_rating'].iloc[j]=df[df['IC']==unique_ic[j]]['Rating'].max()
#   # output['Min_rating'].iloc[j]=df[df['IC']==unique_ic[j]]['Rating'].min()
#   # output['Avg_rating'].iloc[j]=df[df['IC']==unique_ic[j]]['Rating'].mean()

In [25]:
# for j in tqdm(range(len(unique_ic))):
#   # output['IC'].iloc[j]=unique_ic[j]
#   # output['Max_rating'].iloc[j]=df[df['IC']==unique_ic[j]]['Rating'].max()
#   output['Min_rating'].iloc[j]=df[df['IC']==unique_ic[j]]['Rating'].min()
#   # output['Avg_rating'].iloc[j]=df[df['IC']==unique_ic[j]]['Rating'].mean()

In [26]:
# for j in tqdm(range(len(unique_ic))):
#   # output['IC'].iloc[j]=unique_ic[j]
#   # output['Max_rating'].iloc[j]=df[df['IC']==unique_ic[j]]['Rating'].max()
#   # output['Min_rating'].iloc[j]=df[df['IC']==unique_ic[j]]['Rating'].min()
#   output['Avg_rating'].iloc[j]=df[df['IC']==unique_ic[j]]['Rating'].mean()

In [27]:
from tqdm import tqdm
for j in tqdm(range(len(unique_ic[:200]))):
  # output['IC'].iloc[j]=unique_ic[j]
  text=''
  for i in range(len(df[df['IC']==unique_ic[j]])):
    text=text+ df[df['IC']==unique_ic[j]]['Rev_summ'][i] + " " + df[df['IC']==unique_ic[j]]['Review'][i] + '\n'

  cleaned=[]
  for k in text.split('\n'):
      sent=nlp(clean(k))
      cleaned.append(sent)
  # print(len(cleaned))

  article, texts = [], []
  for doc in cleaned:
      for w in doc:
          # if it's not a stop word or punctuation mark, add it to our article
          if not w.is_stop and not w.is_punct and not w.like_num:
              # we add the lematized version of the word
              article.append(w.lemma_)
      texts.append(article)
      article=[]

  bigram = gensim.models.Phrases(texts)
  texts = [bigram[line] for line in texts]

  dictionary = Dictionary(texts)
  corpus = [dictionary.doc2bow(text) for text in texts]

  ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=30, random_state=0)
  topics=ldamodel.print_topics(num_words=20)

  for l in topics:
    string=l[1]
    # print(type(string))
    # print(string)
    string=string.replace(' ','').replace('*','').replace('+','').replace('"','')
    # print(string)
    string = re.sub("\d+", "", string)
    string=string.replace('.',',')
    string=string[1:]
    # print(string)

  output['Keywords'].iloc[j]=string

100%|██████████| 200/200 [34:12<00:00, 10.26s/it] 


In [28]:
sample_output=output.iloc[:200,:]

In [29]:
sample_output.head()

IC  Max_rating  Min_rating  Avg_rating  \
0  B009XD5TPQ           5           2    4.500000   
1  B016MF3P3K           5           1    3.636364   
2  B008DC8N5G           5           1    4.090909   
3  B0089CH3TM           5           1    4.457143   
4  B00AKZWGAC           5           1    4.386935   

                                            Keywords  
0  case,phone,fit,belt,maxx,nice,holster,hold,dro...  
1  edge,screen_protector,screen,bubble,apply,appl...  
2  glass,phone,screen,work,star,replacement,produ...  
3  case,phone,great,fit,like,color,cover,good,lov...  
4  battery,phone,charge,day,case,use,hour,work,ge...

In [30]:
sample_output.shape

(200, 5)

In [31]:
# Sanity check for sample output
print(sample_output['IC'].nunique())
print(sample_output['Max_rating'].nunique())
print(sample_output['Min_rating'].nunique())
print(sample_output['Avg_rating'].nunique())
print(sample_output['Keywords'].nunique())

200
2
5
155
200


In [32]:
# Okay, rows are edited as per expectation
# Lets write it to a json file
sample_output.to_json('sample_output.json')

In [33]:
# Now, writing the data directly to a dataframe is expensive
# Lets try to write the data first to a list/dict and then to Dataframe

dic={}
dic['IC']=[]
dic['Avg_rating']=[]
dic['Max_rating']=[]
dic['Min_rating']=[]
dic['Keywords']=[]

for j in tqdm(range(len(unique_ic[:200]))):
  dic['IC'].append(unique_ic[j])
  dic['Max_rating'].append(df[df['IC']==unique_ic[j]]['Rating'].max())
  dic['Min_rating'].append(df[df['IC']==unique_ic[j]]['Rating'].min())
  dic['Avg_rating'].append(df[df['IC']==unique_ic[j]]['Rating'].mean())

100%|██████████| 200/200 [00:29<00:00,  6.88it/s]


In [35]:
# It is taking the same amount of time.
# To write the output dataframe, it'll take around 10 hours

In [ ]:
# Please use the below code to extract the keywords of any IC


text=''
for i in range(len(df[df['IC']=='PUT IC Here'])):
    text=text+ df[df['IC']=='PUT IC Here']['Rev_summ'][i] + " " + df[df['IC']=='PUT IC Here']['Review'][i] + '\n'

# print(text)

cleaned=[]
for i in text.split('\n'):
    sent=nlp(clean(i))
    cleaned.append(sent)
print(len(cleaned))

article, texts = [], []
for doc in cleaned:
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article
        if not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)
    texts.append(article)
    article=[]

print(len(texts))

bigram = gensim.models.Phrases(texts)
texts = [bigram[line] for line in texts]

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

ldamodel = LdaModel(corpus=corpus, num_topics=1, id2word=dictionary, passes=50, random_state=0)
topics=ldamodel.print_topics(num_words=20)
topics

for l in topics:
    string=l[1]
    # print(type(string))
    # print(string)
    string=string.replace(' ','').replace('*','').replace('+','').replace('"','')
    # print(string)
    string = re.sub("\d+", "", string)
    string=string.replace('.',',')
    string=string[1:]
    print(string)

In [36]:
# The above code will give the corresponding keywords for any product's review